# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [3]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from PIL import Image

/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to

## Reformat Input

In [12]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

## Pass into model 1

In [4]:
def get_video_dimensions(video_path):
    try:
        # Open the video file
        video = cv2.VideoCapture(video_path)

        # Read the first frame
        ret, frame = video.read()

        # Get the dimensions of the frame
        height, width, _ = frame.shape

        return width, height

    except Exception as e:
        print("An error occurred: ", e)

    finally:
        # Release the video capture object
        video.release()

def crop_frame(frame, target_width, target_height):
    # Get the dimensions of the frame
    height, width, _ = frame.shape

    # Calculate the starting point for cropping
    start_x = (width - target_width) // 2
    start_y = (height - target_height) // 2

    # Perform the cropping
    cropped_frame = frame[start_y:start_y+target_height, start_x:start_x+target_width, :]

    return cropped_frame

def pose_predict(image):
    OUTPUT_RATIO = 0.008926701731979847
    OUTPUT_BIAS = 126
    threshold = 0.2/OUTPUT_RATIO+OUTPUT_BIAS
    # Example usage
    # video_path = "posevid.mp4"
    # width, height = get_video_dimensions(video_path)
    # cropped_frame = crop_frame(image, height, height)
    # resized_frame = cv2.resize(image, (368, 368))
    # color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    # frame = np.expand_dims(color_frame, axis=0)
    # frame = frame.astype(np.uint8)
    
    

    # # set input -> invoke -> access output
    # interpreter.set_tensor(input_index, frame)
    # interpreter.invoke()
    # output_data = interpreter.get_tensor(output_index)
    # return output_data


In [5]:
OUTPUT_RATIO = 0.008926701731979847
OUTPUT_BIAS = 126
threshold = 0.2/OUTPUT_RATIO+OUTPUT_BIAS
frame_classifications = []
moving_prob = []

interpreter = tf.lite.Interpreter(model_path="./tflite_models/openpose_mobilenetv0.75_quant_1x368x368x3.tflite")
interpreter.allocate_tensors()

# get_output_details() and get_input_details() return list of dictionaries of tensor details
# keys: name, index, shape, shape_signature, dtype, quantization, ...
# len(input) = len(output) = 1, so access the first element
output = interpreter.get_output_details()
input = interpreter.get_input_details()
output_index = output[0]['index']
input_index = input[0]['index']
AllPoints = []
alloutputdata = []
vid_path = "posevid.mp4"
cap = cv2.VideoCapture(vid_path)
ret, frame = cap.read()
for f in range(0,5):
    width, height = get_video_dimensions(vid_path)
    cropped_frame = crop_frame(frame, height, height)
    frame = cropped_frame
    resized_frame = cv2.resize(frame, (368, 368))
    color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    frame = np.expand_dims(color_frame, axis=0)
    frame = frame.astype(np.uint8)
    interpreter.set_tensor(input_index, frame)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_index)
    alloutputdata.append(output_data)
    indexlst = []
    for i in range(0,46):
        indexrow = []
        for j in range(0,46):
            list = []
            classifications = output_data[0][i][j]
            for k in range(0,57):
                #float32_value = np.float32(classifications[k]) / 255.0
                # if float32_value < 0:
                #     float32_value += 256
                #list.append((float(classifications[k])- OUTPUT_BIAS) * OUTPUT_RATIO)
                list.append(classifications[k])
            classifications = list
            classifications = [round(num, 2) for num in classifications]
            #print(classifications)
            indexrow.append(classifications.index(max(classifications)))
            newarr = np.array(classifications)
                #classifications = [round(num, 4) for num in classifications]
        indexlst.append(indexrow)
            #print(classifications)
    Points = []
    for a in range(0,18):
        coordinates=(-1,-1)
        maxconfidence = -1
        for b in range(0,46):
            for c in range(0,46):
                if indexlst[b][c] == a:
                    #print(output_data[0][b][c][a])
                    if output_data[0][b][c][a] > maxconfidence and output_data[0][b][c][a] >= threshold:
                        maxconfidence = output_data[0][b][c][a]
                        coordinates = (b,c)
        Points.append(coordinates)
    AllPoints.append(Points)
    print(f)
    ret, frame = cap.read()
    # cnt = 0
    # for pt in Points:
    #     if pt[0] != -1:
    #         print("x: "+str(pt[1])+" ","y: "+str(pt[0])+" ","index: "+str(cnt))
    #     cnt+=1

    # print()
    # print()
    # for i in range(0,len(indexlst)):
    #     for j in range(0,len(indexlst[i])):
    #         if indexlst[i][j] <= 17:
    #             print("x: "+str(i)+" ","y: "+str(j)+" ","index: "+str(indexlst[i][j]))

0
1
2
3
4


In [9]:
import sys
#sys.stdout = open('./output.txt', 'w')

'''output1 = alloutputdata[0]
lst = []
for i in range(0, 46):
    for j in range(0, 46):
        for l in range(0, 57):
            print(output1[0][i][j][l])
            lst.append(output1[0][i][j][l])'''

d = alloutputdata[0]
for x in range(0,46):
    for y in range(0,46):
        print(d[0][x][y])
print(np.shape(d))

for pts in AllPoints:
    print(pts)
# sys.stdout.close()
# sys.stdout = sys.__stdout__  # Restore default stdout

    

# Assuming your 1D array is named 'arr'
#arr = output_data[0]
'''array_2d = [[0] * 46 for _ in range(46)]
list = []
row = 0
col = 0
for i in range(len(arr)):
    list.append(arr[i])
    if (i+1) %57 == 0:
        array_2d[row][col] = list
        row+=1
        if row > 45:
            row = 0
            col+=1
        list = []
print(np.shape(array_2d))
print(array_2d[0][0])
print(array_2d[45][45])'''



[127 125 126 125 125 125 126 126 125 126 127 127 125 126 127 127 128 125
 241 126 127 127 128 126 127 125 128 127 126 127 126 126 127 126 126 127
 127 126 124 128 125 128 127 124 125 126 127 125 126 126 126 126 126 126
 125 126 126]
[124 126 126 127 126 125 127 126 126 127 125 125 127 127 126 126 125 127
 237 126 127 126 126 126 126 126 126 126 123 127 124 126 127 126 127 125
 125 125 125 127 125 127 127 124 126 126 125 126 126 127 126 127 127 127
 126 125 125]
[127 127 127 127 126 126 126 126 126 127 126 126 126 125 126 127 125 127
 237 127 128 125 125 126 126 126 126 127 124 127 125 126 127 126 127 124
 126 125 125 127 125 127 126 125 125 126 125 125 125 127 126 126 126 127
 125 125 126]
[126 126 127 127 126 126 126 126 126 126 126 126 126 125 125 126 126 126
 240 127 129 126 126 126 126 126 128 126 126 126 127 126 124 128 126 126
 127 126 126 127 126 127 124 127 127 125 125 125 126 127 125 126 126 126
 125 125 125]
[127 127 127 127 126 126 125 126 127 127 126 126 126 126 126 127 127

'array_2d = [[0] * 46 for _ in range(46)]\nlist = []\nrow = 0\ncol = 0\nfor i in range(len(arr)):\n    list.append(arr[i])\n    if (i+1) %57 == 0:\n        array_2d[row][col] = list\n        row+=1\n        if row > 45:\n            row = 0\n            col+=1\n        list = []\nprint(np.shape(array_2d))\nprint(array_2d[0][0])\nprint(array_2d[45][45])'

In [10]:
#newarr = np.array(AllPoints)
print(np.shape(AllPoints))
print(AllPoints[0])
#print(newarr.shape)
# AllPoints = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Specify the file path and name
file_path = "./pose_output.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
    # Iterate over each sublist in AllPoints
    for sublist in AllPoints:
        # Convert the sublist to a string
        sublist_str = ' '.join(map(str, sublist))
        # Write the sublist string to the file
        file.write(sublist_str + '\n')

# Specify the file path and name
file_path = "./pose_output.txt"

# Initialize the larger list
larger_list = []

# Read the contents of the file
with open(file_path, 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Strip any leading/trailing whitespaces and newline characters
        line = line.strip()
        line = line.replace(", ",",")
        # Split the line by spaces to separate the elements
        elements_str = line.split()
        
        # Ensure the line has exactly 18 elements
        if len(elements_str) == 18:
            # Convert elements to tuples
            elements = [tuple(map(int, elem.strip('()').split(','))) for elem in elements_str]
            
            # Append the elements list to the larger list
            larger_list.append(elements)
new_large = []

for row in larger_list:
    new_row = []
    for pt in row:
        if pt[0] != -1:
            newx = (int)(pt[0]/46*900)
            newy = (int)(pt[1]/46*900)
            pt = (newx,newy) 
            new_row.append(pt)
        else:
            new_row.append((-1,-1))
    new_large.append(new_row)

print(np.shape(new_large))
print(new_large[0])

(5, 18, 2)
[(13, 27), (15, 24), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (40, 15), (-1, -1), (-1, -1), (-1, -1), (12, 27), (-1, -1), (12, 25), (-1, -1)]
(5, 18, 2)
[(254, 528), (293, 469), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (782, 293), (-1, -1), (-1, -1), (-1, -1), (234, 528), (-1, -1), (234, 489), (-1, -1)]


In [19]:

numbers = []
with open("newoutput.txt", "r") as file:
    for line in file:
        number = int(line.strip())
        numbers.append(number)
result = []
result.append(numbers)

newresult = [[0] * 46 for _ in range(46)]
list = []
row = 0
col = 0
for i in range(len(result[0])):
    list.append(result[0][i])
    if (i+1) %57 == 0:
        newresult[row][col] = list
        row+=1
        if row > 45:
            row = 0
            col+=1
        list = []
array_2d = []
array_2d.append(newresult)

print(np.shape(array_2d))
import time
for i in range(0,46):

    for j in range(0,46):
        for k in range(0,57):
            print(array_2d[0][i][j][k])
            time.sleep(0.7)



indexlst = []
for i in range(0,46):
    indexrow = []
    for j in range(0,46):
        list = []
        classifications = array_2d[0][i][j]
        for k in range(0,57):
            #float32_value = np.float32(classifications[k]) / 255.0
            # if float32_value < 0:
            #     float32_value += 256
            #list.append((float(classifications[k])- OUTPUT_BIAS) * OUTPUT_RATIO)
            list.append(classifications[k])
        classifications = list
        classifications = [round(num, 2) for num in classifications]
        #print(classifications)
        indexrow.append(classifications.index(max(classifications)))
        newarr = np.array(classifications)
            #classifications = [round(num, 4) for num in classifications]
    indexlst.append(indexrow)
        #print(classifications)
Points = []
for a in range(0,18):
    coordinates=(-1,-1)
    maxconfidence = -1
    for b in range(0,46):
        for c in range(0,46):
            if indexlst[b][c] == a:
                #print(output_data[0][b][c][a])
                if array_2d[0][b][c][a] > maxconfidence and array_2d[0][b][c][a] >= threshold:
                    maxconfidence = array_2d[0][b][c][a]
                    coordinates = (b,c)
    Points.append(coordinates)

print(Points)

[(27, 13), (-1, -1), (22, 16), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (15, 39), (-1, -1), (-1, -1), (-1, -1), (27, 12), (-1, -1), (25, 12), (-1, -1)]
